# 💻 [Univariate - multi step LSTM](https://youtu.be/xYT65cyNS4M)
### 📚 [AI with Dr. Mo](https://www.youtube.com/c/AIwithDrMo/playlists)


In [1]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

## Create a simple univariate time-series data

In [2]:
df = pd.DataFrame({'x':list(range(17, 329))})
df

,x
0,17
1,18
2,19
3,20
4,21
...,...
307,324
308,325
309,326
310,327


## Shift time-series to get lag and lead steps

In [3]:
num_lags = 3
num_steps = 2

for i in range(num_lags+1):
    df.insert(loc=1, column=f't-{i}' , value=df.iloc[:,0].shift(periods=i))
for j in range(1,num_steps):
        df.insert(loc=num_lags+j+1, column=f't+{j}' , value=df.iloc[:,0].shift(periods=-j))
print(df)

       x    t-3    t-2    t-1  t-0    t+1
0     17    NaN    NaN    NaN   17   18.0
1     18    NaN    NaN   17.0   18   19.0
2     19    NaN   17.0   18.0   19   20.0
3     20   17.0   18.0   19.0   20   21.0
4     21   18.0   19.0   20.0   21   22.0
..   ...    ...    ...    ...  ...    ...
307  324  321.0  322.0  323.0  324  325.0
308  325  322.0  323.0  324.0  325  326.0
309  326  323.0  324.0  325.0  326  327.0
310  327  324.0  325.0  326.0  327  328.0
311  328  325.0  326.0  327.0  328    NaN

[312 rows x 6 columns]


## Create X and y from the above shifted time series

In [4]:
df.dropna(inplace=True)

X = df.iloc[:,1:num_lags+1].values
y = df.iloc[:,num_lags+1:].values

print(X.shape)
print(y.shape)

(308, 3)
(308, 2)


In [6]:
# reshape from [samples, num_lags] into [samples, num_lags, num_feat]
num_feat = 1
X = X.reshape((X.shape[0], X.shape[1], num_feat))

## Train model and predict multi step out values

In [7]:
# define model
model = Sequential()
model.add(LSTM(40, activation='relu', input_shape=(num_lags, num_feat)))
model.add(Dense(num_steps))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([300, 301, 302])
x_input = x_input.reshape((1, num_lags, num_feat))
y_pred = model.predict(x_input, verbose=0)
print(y_pred)

2022-07-24 11:31:16.485685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[[302.98962 303.95218]]
